In [1]:
!pip install crewai_tools python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.8 MB/s

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API configuration from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE", "https://api.mistral.ai/v1")
OPENAI_MODEL_NAME = os.environ.get("OPENAI_MODEL_NAME", "mistral-small")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file or environment variables.")

# Set them in os.environ as crewai might expect them there directly
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE
os.environ["OPENAI_MODEL_NAME"] = OPENAI_MODEL_NAME

#**Agent Planner**

Plan how blog is actually will be writen

In [4]:
planner=Agent(
    role="Content Plnner",
    goal=" plan  enagaging and fact based content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True

)

#**Agent Writer**


In [5]:
writer=Agent(
    role="Content Writer",
    goal="write a meaningfull and factbased and"
    " accurate opinion peice about this {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

#**Agent Editor**


In [6]:
editor=Agent(
    role="Content Editor",
    goal="Edit the  give blog to allign"
          "with the Organization style",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

##**Creating Task**
- Define task, and provide them with `description`, `expected output` and `agent`.


##**Taks : Plan**

In [7]:
plan=Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output=
        "A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",

    agent=planner
)


In [8]:
plan=Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
        expected_output="A well-written blog post in markdown format, "
                        "ready for publication, "
                        "each section should have 2 or 3 paragraphs.",
    agent=editor
)


##**Task: Write**##


In [9]:
write=Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer
)

##**Writer**

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
        expected_output="A well-written blog post in markdown format, "
                        "ready for publication, "
                        "each section should have 2 or 3 paragraphs.",
    agent=editor
)

##**Crteating the Crew Of Agent**


In [17]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=0
)

## **Running the Crew**

In [18]:
result = crew.kickoff(inputs={"topic":"Pakistan Vs India "})













# Agent: Content Editor
## Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.


# Agent: Content Editor
## Final Answer: 
Introduction:

It is essential to have a well-written and informative introduction that grabs the reader's attention and sets the tone for the rest of the blog post. The introduction should provide a brief overview of the topic and what the reader can expect to learn.

---

Section 1: The Importance of Journalistic Best Practices

As a content editor, following journalistic best practices is crucial to ensure that the blog post is unbiased, accurate, and provides value to the reader. Adhering to these practices not only helps to establish credibility but also ensures that the information presented is trustworthy.

---

Section 2: Balanced Viewpoints

Providing balanced viewpoints when offering opinions or assertions is essential to creating a well-rounded and informative blog post. This involves presenting bo

In [22]:
from IPython.display import Markdown
Markdown(result.raw)

Introduction:

The long-standing rivalry between Pakistan and India is a topic that has been discussed and analyzed for decades. From politics to sports, the two countries have a history of competition and conflict. In this blog post, we will explore the complex relationship between Pakistan and India, focusing on journalistic best practices, balanced viewpoints, and avoiding controversial topics.

Section 1: The Importance of Journalistic Best Practices

When writing about the topic "Pakistan vs India," it is crucial to follow journalistic best practices to ensure that the information presented is unbiased, accurate, and provides value to the reader. Adhering to these practices not only helps to establish credibility but also ensures that the information presented is trustworthy.

For example, using reliable sources to gather information, double-checking facts, and providing context to help the reader understand the topic better are all essential journalistic best practices. Additionally, avoiding sensationalism, bias, and speculation is crucial to maintaining a professional and unbiased tone. At [Organization], we prioritize fact-checking and ensuring that our sources are trustworthy. Our goal is to provide our readers with accurate and reliable information that they can use to make informed decisions.

Section 2: Balanced Viewpoints

Providing balanced viewpoints when offering opinions or assertions is essential to creating a well-rounded and informative blog post. This involves presenting both sides of an argument and allowing the reader to make their own informed decision.

For instance, when discussing the political tensions between Pakistan and India, it is essential to provide context and background information about the disputes, as well as the positions of both countries. At [Organization], we believe in presenting a balanced viewpoint, even when discussing complex and sensitive topics. Our goal is to help our readers understand the nuances of an issue and make informed decisions.

Section 3: Avoiding Controversial Topics

While it is essential to address relevant and timely topics, avoiding major controversial topics or opinions is often necessary to maintain a professional and unbiased tone. This helps to ensure that the blog post is accessible to a wider audience and is not likely to offend or alienate readers.

For example, when discussing the cricket rivalry between Pakistan and India, it is best to focus on the sport itself and the skills of the players, rather than making controversial statements about nationalism or patriotism. At [Organization], we prioritize inclusivity and respect for all viewpoints. Our goal is to create content that is accessible and engaging for a diverse audience.

Section 4: Aligning with the Organization's Style

As a content editor, it is crucial to ensure that the blog post aligns with the organization's style and tone. This involves using consistent formatting, language, and branding throughout the post.

For instance, if the organization's style guide emphasizes a conversational tone, it is essential to use language and formatting that reflects this style. At [Organization], we prioritize a friendly and approachable tone that resonates with our audience. Our goal is to create content that is easy to read and engaging for our community.

Conclusion:

In conclusion, when writing about the topic "Pakistan vs India," it is essential to follow journalistic best practices, provide balanced viewpoints, avoid controversial topics, and align with the organization's style. By doing so, you can create a well-written and informative blog post that provides value to the reader and establishes credibility for your organization.

Remember to always double-check your facts, provide context, and avoid sensationalism or bias. By doing so, you can create a blog post that is both informative and unbiased, providing value to your readers and establishing your organization as a trusted source of information. At [Organization], we are committed to upholding these standards in all of our content.